# Search for replicas

In [ ]:
%aiida
from apps.mcmasters.replicawork import ReplicaWorkchain
from aiida.work.process import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview
import time
import ase.io
import urlparse
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return '<img width="400px" src="data:image/png;base64,{}" title="">'.format(th)
def html_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

In [ ]:
qb = QueryBuilder()

qb.append(
    WorkCalculation,
    #filters={
    #    'attributes._finished': {'==': True}
    #},
    tag='wc'
)

qb.append(
    Cp2kCalculation,
    filters={
        'label': {'==': 'replica_geo_opt'},
        'description': {'like': 'replica_%'}
    },
    tag='cp2k',
    output_of='wc',
    project='description'
)

qb.append(
    StructureData,
    output_of='cp2k',
    project='*'
)

qb.append(
    ParameterData,
    output_of='cp2k',
    filters={
        'attributes.exceeded_walltime': {'==': False}
    },
    project=['attributes.energy', 'ctime']
)

qb.append(
    ParameterData,
    input_of='cp2k',
    filters={
        'attributes': {'has_key': 'MOTION'}
    },
    project='*'
)

qb.order_by({'cp2k': {'ctime': 'asc'}})

r = qb.all()

In [ ]:
def replica_parse_desc(r):
    import re
    from collections import OrderedDict
    pattern = r'replica_([\w\-]+)_([\d\.]+)'
    names_count = OrderedDict()
    for rep in r:
        #print '{} '.format(rep[0])
        #print ' ==> {}'.format(re.findall(pattern, rep[0]))
        name, target = re.findall(pattern, rep[0])[0]
        try:
            names_count[name].append((target, rep[1], rep[2], rep[4], rep[3]))
        except KeyError:
            names_count[name] = [(target, rep[1], rep[2], rep[4], rep[3]), ]
        #print (name, target)
    return names_count

def get_replica_distance(s1, s2):
    a1 = s1.get_positions()
    a2 = s2.get_positions()
    return np.linalg.norm(a1-a2)

In [ ]:
print 'Parsing replicas...'
replica = replica_parse_desc(r)
keys, date = replica.keys(), [str(replica[rep][0][4]) for rep in replica.keys()]
print 'Found replicas: '
for x in zip(keys, date):
    print '{} - {}'.format(x[0], x[1])

drop_replica = ipw.Dropdown(options = keys,
                            description = 'Replica: ',
                            index = len(keys)-1)
display(drop_replica)

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

text_name = ipw.Text(placeholder='Replica name',
                     description='Replica name',
                     style=style, layout=layout)

def on_submit(b):
    replica_name = drop_replica.value
    show_th = show_thumbnails(replica[replica_name])
    with output_thumbnails:
        clear_output()
        #x.get_attr('FORCE_EVAL')['SUBSYS']['COLVAR']
        html = 'COLVAR: {}'.format(str(replica[replica_name][0][3].get_attr('FORCE_EVAL')['SUBSYS']['COLVAR']))
        html += show_th['html']        
        display(ipw.HTML(html))

    with output_plot:
        plt.figure(figsize=(10, 5))
        plt.ylabel('Energy/hartree')
        plt.xlabel('Collective variable/angstrom')
        plt.plot(show_th['plot'][0], show_th['plot'][1], 'o-')
        plt.show()
        
    with output_header:
        clear_output()
        html = '<h2>{}</h2><br>'.format(replica_name)
        display(ipw.HTML(html))

output_thumbnails = ipw.Output()
output_plot = ipw.Output()
output_header = ipw.Output()
btn_submit = ipw.Button(description="Show")
btn_submit.on_click(on_submit)
display(btn_submit, output_header, output_plot, output_thumbnails)

In [ ]:
def show_thumbnails(replica):
    from copy import deepcopy
    good_replica = []
    plot_energy = []
    plot_colvar = []
    
    html = '<table>'
    for i, rep in enumerate(replica):
        with output_header:
            clear_output()
            print '{}/{}'.format(i+1, len(replica))
        struc = rep[1].get_ase()

        # replace the target atoms with O
        t = deepcopy(struc)
        t1, t2 = map(int, (rep[3].get_attr('FORCE_EVAL')['SUBSYS']['COLVAR']['DISTANCE']['ATOMS']).split())
        t[t1-1].symbol = 'O'
        t[t2-1].symbol = 'O'
        d = t[t1-1].position-t[t2-1].position

        if i%2 == 0:
            html += '<tr>'
        html += '<td><img width="400px" src="data:image/png;base64,{}" title="">'.format(render_thumbnail(t))
        good_replica.append([rep[0], struc, rep[2]])
        try:
            d2next = get_replica_distance(good_replica[i-1][1], struc)
        except IndexError:
            d2next = '-'
        html += '<p><b>Target: {}</b> ({})<br> <b>Energy:</b> {}<br> <b>d2next:</b> {}</p>'.format(rep[0], np.linalg.norm(d), rep[2], d2next)
        html += '<p><a target="_blank" href="./export_structure.ipynb?uuid={}">Export Structure</a></p><td>'.format(rep[1].uuid)
    
        if i%2 == 1:
            html += '</tr>'
            
        plot_energy.append(rep[2])
        plot_colvar.append(np.linalg.norm(d))

    html += '</tr>'
    html += '</table>'
    return {
        'html': html,
        'plot': (plot_colvar, plot_energy)
    }

In [ ]:
def show_ngl(replica): # don't
    the_views = []
    for i, rep in enumerate(replica):
        the_views.append(ipw.Output())
        display(the_views[-1])
        with the_views[-1]:
            clear_output()
            display(nglview.show_ase(rep[1].get_ase()))

        print rep[0], rep[1], rep[2]
    